In [2]:
import pandas as pd
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

In [12]:
df = pd.read_csv("data/100x100.csv")
df_2 = pd.read_csv("data/100x100_2.csv")


In [ ]:
def spectral_regularization_model(x,df):
    data_matrix = df.pivot(index='u_id', columns='a_id', values='score').fillna(0)
    delta = df[['u_id', 'a_id', 'score']] #set of all known ratings
    num_users = data_matrix.shape[0]
    num_anime = data_matrix.shape[1]

    R=cp.Variable(shape=(num_users,num_anime)) #predicted rating matrix
    
    user_id_to_index = {} #helper function
    for idx in range(len(data_matrix.index)):
        u_id = data_matrix.index[idx]
        user_id_to_index[u_id] = idx
    
    anime_id_to_index = {} #helper function
    for idx in range(len(data_matrix.columns)):
        a_id = data_matrix.columns[idx]
        anime_id_to_index[a_id] = idx

    training_error=[]
    for ui in range(len(delta)):
        u=user_id_to_index[int(delta.at[ui,'u_id'])]
        i=anime_id_to_index[int(delta.at[ui,'a_id'])]
        training_error.append((R[u][i]-int(delta.at[ui,'score']))**2) #(R_ui-A_ui)^2

    training_error_sum=cp.sum(training_error) #sum((R_ui-A_ui)^2)


    obj=cp.Minimize(0.5*training_error_sum+ x*cp.norm(R,"nuc"))
    prob=cp.Problem(obj)
    prob.solve()

    print("status:", prob.status)
    if prob.status == cp.OPTIMAL:
        print("The predicted matrix is ", R.value) 
    
    return np.matrix(R.value)




    



In [35]:
R=spectral_regularization_model(10,df_2)
type(R)

status: optimal
The predicted matrix is  [[5.71857821 6.82050028 6.71242977 ... 5.55329474 6.02466116 5.26576902]
 [7.55421696 8.78681574 8.25059731 ... 6.66562812 7.0994061  6.890764  ]
 [8.02570425 9.17688937 8.40898644 ... 6.73118548 7.21093343 7.29996729]
 ...
 [6.54580626 7.84823034 7.77063422 ... 6.436666   6.97117337 6.02930679]
 [5.11801093 6.14928334 6.16650267 ... 5.15510351 5.69213442 4.73357743]
 [3.70289773 4.72577794 5.22470298 ... 4.57400514 5.16260543 3.52573554]]


numpy.matrix

In [34]:
def RMSE(R,df):
    data_matrix = df.pivot(index='u_id', columns='a_id', values='score').fillna(0)
    delta = df[['u_id', 'a_id', 'score']]
    N=len(delta)

    user_id_to_index = {} #helper function
    for idx in range(len(data_matrix.index)):
        u_id = data_matrix.index[idx]
        user_id_to_index[u_id] = idx
    
    anime_id_to_index = {} #helper function
    for idx in range(len(data_matrix.columns)):
        a_id = data_matrix.columns[idx]
        anime_id_to_index[a_id] = idx

    LSE=[]
    for ui in range(len(delta)):
            u = user_id_to_index[int(delta.at[ui,'u_id'])]
            i = anime_id_to_index[int(delta.at[ui,'a_id'])]
            LSE.append((R[u,i]-int(delta.at[ui,'score']))**2)
    RMSE=np.sqrt(np.sum(LSE)/N)

    return RMSE

In [ ]:
RMSE(R,df) #RMSE spectral_regularization_model df 1

np.float64(1.307188529413347)

In [36]:
RMSE(R,df_2) #RMSE spectral_regularization_model df 2

np.float64(1.6065010288742316)